<a href="https://colab.research.google.com/github/ddeweerd/VAE_Transcriptomics/blob/main/VAE_module_extractation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introduction
Welcome to the code for extracting gene modules based on signal extraction from multifactorial data. In this context, we introduce the Variational Autoencoder (VAE) method, as outlined in the work by de Weerd et al. (2023). The VAE offers a multi-scale representation capable of encoding cellular processes, encompassing factors from cell types to gene-gene interactions.

In the study by de Weerd et al., we harnessed the power of this VAE to predict gene expression changes in 25 independent disease datasets. Leveraging the learned healthy representations within the VAE, we successfully extracted and decoded disease-specific signals residing within the latent space. As a result, we achieved substantial enrichment of genes directly associated with the diseases under investigation.

This notebook also introduces the accompanying R package, VAEanalysis, along with illustrative code that showcases how to extract, amplify, and decode disease signals from independent case-control datasets.



First, we install the R package from GitHub. We are also using the dplyr and keras packages.

In [ ]:
devtools::install_github("ddeweerd/VAEanalysis", force=TRUE)



png         (NA -> 0.1-8 ) [CRAN]
here        (NA -> 1.0.1 ) [CRAN]
RcppTOML    (NA -> 0.2.2 ) [CRAN]
reticulate  (NA -> 1.39.0) [CRAN]
config      (NA -> 0.3.2 ) [CRAN]
tfautograph (NA -> 0.3.2 ) [CRAN]
tfruns      (NA -> 1.5.3 ) [CRAN]
zeallot     (NA -> 0.1.0 ) [CRAN]
tensorflow  (NA -> 2.16.0) [CRAN]
keras       (NA -> 2.15.0) [CRAN]


Installing 10 packages: png, here, RcppTOML, reticulate, config, tfautograph, tfruns, zeallot, tensorflow, keras

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpAgw9z1/remotes13a520d3655/ddeweerd-VAEanalysis-248b9cb/DESCRIPTION’ ... OK
* preparing ‘VAEanalysis’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘VAEanalysis_0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(VAEanalysis)
library(dplyr)
library(keras)

First, we have a look at the transcriptomics data. The model assumes each row of the input matrix to correspond to a specific gene, with the specific gene names stored in the genename variable. The data should be a matrix with gene counts normalized such that ln(count + 1).

Let's look at the data and gene names. In this example, we look at the gene expression data from active leasons in multiple sclerosis (GSE138614, Elkjaer et al., 2019).

In [ ]:
patient_counts[1:10, 1:10] # Show some of the data
genenames[1:10] # And the corresponding gene names

Next, we load the VAE model from de Weerd et al. Since we are interested in the latent space to extract the disease signal, the encoder and the decoder are loaded separately.

In [ ]:
VAE_builder <- reticulate::import_from_path(module = "VAE_builder", path = system.file(package = "VAEanalysis"))
utils_VAE <- reticulate::import_from_path(module = "utils_VAE", path = system.file(package = "VAEanalysis"))

In [ ]:
encoder <- keras::load_model_hdf5(system.file("extdata", "encoder.h5", package = "VAEanalysis"))
decoder <- keras::load_model_hdf5(system.file("extdata", "decoder.h5", package = "VAEanalysis"))


To extract the disease vector, we need to compress the patient and control data into the latent space of the VAE. The mean of the latent control vectors is then subtracted from the respective vector of the patients, as denoted by Equation (1) in the manuscript. We use the function get_condition_latent_vector to get this disease vector.



In [ ]:
latent_vector <- get_condition_latent_vector(patient_counts = patient_counts,
                                             control_counts = control_counts,
                                             encoder)

By increasing the disease vector in the latent space by a factor eta (default value 3), we can decompress the disease vector using the decoder, as described in Equation (2) in de Weerd et al. 2023. Furthermore, to compare the disease latent vector with a background, we generate and decode 1,000 random vectors. The genes of the decompressed disease vector are compared with the random background using a ranking (as described in Equation (3) in de Weerd et al.). These functionalities are embedded into the get_gene_ranks function.

In [ ]:
ranks <- get_gene_ranks(latent_vector = latent_vector,
                                 boost_factor = 3,
                                 decoder = decoder,
                                 n_comparison = 1000)

Lastly, we can extract the top genes of the ranking. These genes are to be considered the disease module, and can serve as a basis for further analysis such as the extraction of the biggest connected component.

In [ ]:
genenames[ranks < 50] %>% as.data.frame
